## Leonardo E. Wajnsztok
## Matrícula 1312737

### Zeros de funções

#### Implementar e testar os seguintes métodos para zero de funções:

1) Regular falsi para determinar zero de uma função real f:R->R

2) Métode de Broyden I e II para determinar zero de uma função multivariada F:Rˆn->Rˆn.

3) Implementar uma função que desenhe as curvas de nível zero de f:R^2->R e g:R^2->R.

4) Considere f(x,y)= x^2 + y^2 - 1.0 e g(x,y) = x*y - 1.0. Teste a implementação do item 3 e descubra usando os métodos implementados no item 2 as raízes do sistema:

f(x,y) = 0

g(x,y)= 0 

In [9]:
import numpy as np
import math
from scipy import dot
from scipy.linalg import inv
from __future__ import division

import matplotlib.pyplot as plt
%matplotlib inline

In [10]:

def regular_falsi(f,a,b,N):
    i = 0
    while(i < N):
        c = (b*f(a) - a*f(b))/(f(a) - f(b))
        if f(c) == 0:
            return c, i
        if f(a)*f(c) < 0:
            b = c
        else:
            a = c
        i += 1
    return c, i

def f(x):
    return x**2-x

result, steps = regular_falsi(f,0.5,2.0,100)
print(result, steps)

1.0 52


In [11]:
def update(C, F, x, B, f_at_x):
    """
    The updating logic behind Broyden's method.
    """
    # f_at_x = F(x)
    p = - dot(C, f_at_x)
    x = x + p
    new_f_at_x = F(x)
    y = new_f_at_x - f_at_x
    C_dot_y = dot(C, y)
    p_dot_C = dot(p.T, C)
    C = C + dot((p - C_dot_y), p_dot_C) / (dot(p.T, C_dot_y))
    f_at_x = new_f_at_x
    B = inv(C)
    return (C, F, x, B, f_at_x)


def broyden_gen(F, x, B, C, f_at_x, n=5):
    """
    Generator implementing Broyden's update.
    """
    for i in range(n):
        C, F, x, B, f_at_x = update(C, F, x, B, f_at_x)
        yield (x, B, f_at_x, C)

In [12]:

def f(x,y):
    return x*2 + y*2 - 1

def g(x,y):
    return x*y - 1

def reta(t,px,py,qx,qy):
    return px+t*(qx-px), py+t*(qy-py)

def bissecao(f,c,px,py,qx,qy,N):
    a = 0.0
    b = 1.0
    ax,ay = reta(a,px,py,qx,qy)
    bx,by = reta(b,px,py,qx,qy)
    m = (a+b)/2.0
    mx,my = reta(m,px,py,qx,qy)
    fa = f(ax,ay)-c
    fb = f(bx,by)-c
    fm = f(mx,my)-c
    for i in range(N):
        if (fa*fm < 0.0):
            b  = m
            fb = fm
        elif (fb*fm < 0.0):
            a  = m
            fa = fm
        else:
            break
        m  = (a+b)/2.0
        mx,my = reta(m,px,py,qx,qy)
        fm = f(mx,my)-c
    return mx,my
    


def triangleintersect(f,c,ax,ay,bx,by,cx,cy,N):
    fa = f(ax,ay)-c
    fb = f(bx,by)-c
    fc = f(cx,cy)-c
    p  = []
    if (fa*fb <= 0.0):
        p.append(bissecao(f,c,ax,ay,bx,by,N))
    if (fb*fc <= 0.0):
        p.append(bissecao(f,c,bx,by,cx,cy,N))
    if (fc*fa <= 0.0):
        p.append(bissecao(f,c,cx,cy,ax,ay,N))
    return p


def levelcurves(f,g,cmin,cmax,xmin,xmax,ymin,ymax,lc,m,n,NB):
    deltax = (xmax - xmin)/(m-1)
    deltay = (ymax - ymin)/(n-1)
    deltac = (cmax - cmin)/(lc-1)
    for k in range(lc):
        l = []
        l2 = []
        c = cmin + k*deltac
        for i in range(n):
            x0 = xmin + i*deltax
            x1 = x0 + deltax
            for j in range(m): 
                y0 = ymin + j*deltay
                y1 = y0 + deltay
                aresta = triangleintersect(f,c,x0,y0,x1,y0,x1,y1,NB)
                aresta2 = triangleintersect(g,c,x0,y0,x1,y0,x1,y1,NB)
                if (aresta != []):
                    l.append(aresta)
                if (aresta2 != []):
                    l2.append(aresta2)
                aresta = triangleintersect(f,c,x0,y0,x1,y1,x0,y1,NB)
                aresta2 = triangleintersect(g,c,x0,y0,x1,y1,x0,y1,NB)
                if (aresta != []):
                    l.append(aresta)
                if (aresta2 != []):
                    l2.append(aresta2)
        for i in range(len(l)):
            px = []
            py = [] 
            for j in range(len(l[i])):
                px.append(l[i][j][0])
                py.append(l[i][j][1])
            plt.plot(px,py,color='blue')
        for i in range(len(l2)):
            px = []
            py = [] 
            for j in range(len(l2[i])):
                px.append(l2[i][j][0])
                py.append(l2[i][j][1])
            plt.plot(px,py,color='red')
    plt.xlim(xmin,xmax)
    plt.ylim(ymin,ymax)
    plt.show()